In [1]:
%use kandy
%use dataframe

Define basic properties for simulation.
'startDate' and 'endDate' could be changed because of all assets' availability.

In [2]:
import portfolio.rebalancer.strategy.HAAStrategy
import java.time.ZoneId
import java.time.ZonedDateTime

val strategy = HAAStrategy()
val initialBudget = 10000.0
val startDate = ZonedDateTime.of(1970, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())
val endDate = ZonedDateTime.of(2024, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())

Load all the CSV files including price data of various ETFs

In [3]:
import portfolio.rebalancer.io.PriceDataManager

val csvFilesDir = System.getProperty("user.dir") + "../csv"
val allPrices = PriceDataManager(baseDir = csvFilesDir).allPrices

Simulate HAA strategy in as longest period where all the assets are available as possible

In [4]:
import portfolio.rebalancer.StrategySimulator
import portfolio.rebalancer.dto.SymbolPricesByDate

val strategySimulator = StrategySimulator()
val result = strategySimulator.simulate(
    budget = initialBudget,
    strategy = strategy,
    allTimeSymbolPrices = SymbolPricesByDate(value = allPrices),
    startDate = startDate,
    endDate = endDate,
)

Start simulating from 2007-05-31T06:00+09:00[Asia/Seoul] with 10000.0 USD
datesToReBalance=[2008-05-31T06:00+09:00[Asia/Seoul], 2008-07-01T06:00+09:00[Asia/Seoul], 2008-07-30T06:00+09:00[Asia/Seoul], 2008-08-30T06:00+09:00[Asia/Seoul], 2008-09-30T06:00+09:00[Asia/Seoul], 2008-10-30T06:00+09:00[Asia/Seoul], 2008-11-29T06:00+09:00[Asia/Seoul], 2008-12-30T06:00+09:00[Asia/Seoul], 2009-01-30T06:00+09:00[Asia/Seoul], 2009-02-28T06:00+09:00[Asia/Seoul], 2009-03-28T06:00+09:00[Asia/Seoul], 2009-04-28T06:00+09:00[Asia/Seoul], 2009-05-28T06:00+09:00[Asia/Seoul], 2009-06-27T06:00+09:00[Asia/Seoul], 2009-07-28T06:00+09:00[Asia/Seoul], 2009-08-28T06:00+09:00[Asia/Seoul], 2009-09-29T06:00+09:00[Asia/Seoul], 2009-10-28T06:00+09:00[Asia/Seoul], 2009-11-28T06:00+09:00[Asia/Seoul], 2009-12-29T06:00+09:00[Asia/Seoul], 2010-01-28T06:00+09:00[Asia/Seoul], 2010-02-27T06:00+09:00[Asia/Seoul], 2010-03-27T06:00+09:00[Asia/Seoul], 2010-04-28T06:00+09:00[Asia/Seoul], 2010-05-28T06:00+09:00[Asia/Seoul], 2010-06-

Total profit rate

In [5]:
"${(result.timeSeriesData.last().second / initialBudget - 1.0) * 100}% between ${result.timeSeriesData.first().first} and ${result.timeSeriesData.last().first}"

301.87340410999985% between 2007-05-31T06:00+09:00[Asia/Seoul] and 2023-12-30T06:00+09:00[Asia/Seoul]

Plot a line graph showing total asset over time

In [6]:
import java.time.format.DateTimeFormatter

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

val dates = result.timeSeriesData.map { it.first.format(formatter) }
val assets = result.timeSeriesData.map { it.second }

mapOf(
    "date" to dates,
    "asset" to assets,
).plot {
    layout { 
        title = "Total asset over time"
        size = 1200 to 400
     }
    line {
        x(dates)
        y(assets)
        color = Color.PURPLE
        width = 1.0
    }
}

Drawdowns over time

In [7]:
import portfolio.rebalancer.DrawdownCalculator

val dates = result.timeSeriesData.map { it.first.format(formatter) }.drop(1)
val drawdowns = DrawdownCalculator().calculateDrawdowns(result.timeSeriesData.map { it.second })
val mdds = drawdowns.scan(0.0) { acc, x -> minOf(acc, x) }.drop(1)

mapOf(
    "date" to dates + dates,
    "drawdown" to drawdowns + mdds,
    "line" to List(dates.size) { "Drawdown" } + List(dates.size) { "MDD" },
).plot {
    groupBy("line") {
        line {
            x("date")
            y("drawdown")
            color("line")
        }
    }
    layout {
        title = "Drawdown and MDD over time"
        size = 1200 to 400
    }
}